In [2]:
!pip install -q transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import random
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup,
    AutoTokenizer,
    AutoModelWithLMHead
)


def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)

set_seed(42)

tokenizer = AutoTokenizer.from_pretrained('mrm8488/t5-base-finetuned-common_gen')
t5_model = AutoModelWithLMHead.from_pretrained('mrm8488/t5-base-finetuned-common_gen')

/usr/local/lib/python3.9/dist-packages/transformers/models/auto/modeling_auto.py:1295: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [ ]:
import pandas as pd
df = pd.read_csv("/content/comb.csv")

In [ ]:

no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        
        "params": [p for n, p in t5_model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
    {
        "params": [p for n, p in t5_model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]

optimizer = torch.optim.RAdam(t5_model.parameters())


In [ ]:
df = df.drop(columns=['Unnamed: 0'])
df = df.dropna()
o = []
for i,j in df.iterrows():

  o.append((j['questions'],j['answers']))


true_false_adjective_tuples = o

In [ ]:
import datetime
model = t5_model
model.train()
data  = o
epochs = 100
device = 'cuda'
model.to(device)
for epoch in range(epochs):
  print ("epoch ",epoch)
  for input,output in data:
    
    input_sent = "Question: "+input+ " </s>"
    # print(output)
    ouput_sent = output+" </s>"

    tokenized_inp = tokenizer.encode_plus(input_sent,  max_length=500, pad_to_max_length=False,return_tensors="pt")
    tokenized_output = tokenizer.encode_plus(ouput_sent, max_length=500, pad_to_max_length=False,return_tensors="pt")


    input_ids  = tokenized_inp["input_ids"].to(device, dtype=torch.long)
    attention_mask = tokenized_inp["attention_mask"].to(device, dtype=torch.long)
   
    lm_labels= tokenized_output["input_ids"].to(device, dtype=torch.long)
    decoder_attention_mask=  tokenized_output["attention_mask"].to(device, dtype=torch.long)

    output = model(input_ids=input_ids, labels=lm_labels,decoder_attention_mask=decoder_attention_mask,attention_mask=attention_mask)
    
    loss = output[0]

    loss.backward()
    optimizer.step()

    optimizer.zero_grad()

model.save_pretrained("q&amodel")